In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pingouin

In [ ]:
data = pd.read_csv("MRI_study.csv")
data.set_index("id", inplace=True)
data

In [ ]:
data["Number of changes (Hound)"].sum()

In [ ]:
from pandas.core.internals.blocks import ensure_block_shape
melted = data.melt(id_vars=['Group','Hospital'], value_vars=['Accuracy (Ram, before AI support)', 'Accuracy (Ram, without AM)', 'Accuracy (Ram, with AM)',
                                                  'Accuracy (Hound, before AI support)', 'Accuracy (Hound, without AM)', 'Accuracy (Hound, with AM)'])
melted

In [ ]:
from pandas.core.internals.blocks import ensure_block_shape
#melted = data.melt(id_vars=['Group'], value_vars=['Accuracy (Ram, before AI support)', 'Accuracy (Ram, without AM)', 'Accuracy (Ram, with AM)',
#                                                  'Accuracy (Hound, before AI support)', 'Accuracy (Hound, without AM)', 'Accuracy (Hound, with AM)'])
#melted_ren = data.rename(columns={'Accuracy (Hound, after AI support)': 'Accuracy (Hound)'}).melt(id_vars=['Group'], value_vars=['Accuracy (Ram)', 'Accuracy (Hound)'])
melted.columns = ["AI Expertise", "Hospital", "variable", "value"]
melted["Group"] = np.nan

for i in range(melted.shape[0]):
  if "Ram" in melted.loc[i,"variable"]:
    melted.loc[i,"Group"] = "Ram"
  else:
    melted.loc[i,"Group"] = "Hound"
  if "before AI support" in melted.loc[i,"variable"]:
    melted.loc[i,"variable"] = "HD1"
  elif "without AM" in melted.loc[i,"variable"]:
    melted.loc[i,"variable"] = "(HD1-)AI-FHD"
  else:
    melted.loc[i,"variable"] = "(HD1-)AI-XAI-FHD"
melted.columns = ["AI Expertise", "Hospital", "Protocol Type", "Accuracy", "Group"]
melted["Protocol"] = np.nan
for i in range(melted.shape[0]):
  if melted.loc[i, "Protocol Type"] == "HD1":
    melted.loc[i, "Protocol"] = "HD1"
  elif melted.loc[i, "Protocol Type"] == "(HD1-)AI-FHD":
    melted.loc[i, "Protocol"] = "HD1-AI-FHD \n(Hound)" if melted.loc[i, "Group"] == "Hound" else "AI-FHD \n(Ram)"
  else:
    melted.loc[i, "Protocol"] = "HD1-AI-XAI-FHD \n(Hound)" if melted.loc[i, "Group"] == "Hound" else "AI-XAI-FHD \n(Ram)"
melted = melted.dropna().sort_values(by=["Group", "Protocol"])
melted

In [ ]:
data_aggr = melted.copy()
data_aggr = data_aggr[data_aggr["Protocol"] != "HD1"]
for i in data_aggr.index:
  if data_aggr.loc[i,"Group"] == "Ram":
    data_aggr.loc[i,"Protocol"] = "Hound vs Ram"
    data_aggr.loc[i,"Expertise"] = "THC"
  elif data_aggr.loc[i,"Group"] == "Hound" and data_aggr.loc[i,"Protocol"] != "HD1":
    data_aggr.loc[i,"Protocol"] = "Hound vs Ram"
    data_aggr.loc[i,"Expertise"] = "SHC"
melted_temp = melted
melted = pd.concat([melted, data_aggr])

In [ ]:
def calculate_diff(df):
  mean_df = pd.DataFrame(
      np.zeros( (len(df["Protocol"].unique())) ),
      index = df["Protocol"].unique(),
      columns = ["Mean"]
      )
  print(df["Protocol"].unique())
  for i in df["Protocol"].unique():
    filter1 = (df["AI Expertise"] == "THC") & (df["Protocol"] == i)
    filter2 = (df["AI Expertise"] == "SHC") & (df["Protocol"] == i)
    val = df[filter1]["Accuracy"].mean() - df[filter2]["Accuracy"].mean()
    mean_df.loc[i] = val
  return mean_df

def calculate_err(df_std, df_count):
  err_df = pd.DataFrame(
      np.zeros( (len(df_std["Protocol"].unique())) ),
      index = df_std["Protocol"].unique(),
      columns = ["Error"]
      )
  for i in df_std["Protocol"].unique():
    filter1 = (df_std["AI Expertise"] == "THC") & (df_std["Protocol"] == i)
    count1 = df_count[filter1]["Accuracy"].count()
    tmp1 = (df_std[filter1]["Accuracy"].std()**2) * (count1 - 1)

    filter2 = (df_std["AI Expertise"] == "SHC") & (df_std["Protocol"] == i)
    count2 = df_count[filter2]["Accuracy"].count()
    tmp2 = (df_std[filter2]["Accuracy"].std()**2) * (count2 - 1)

    varp = tmp1 + tmp2/(count1 + count2 - 2)
    print(np.sqrt(varp/count1 + varp/count2))
    err_df.loc[i] = 1.96*np.sqrt(varp/(count1 + count2))
  return err_df


means = calculate_diff(melted)
errors = calculate_err(melted, melted)
melted.groupby(by=["Protocol","AI Expertise", "Hospital"]).mean()
errors.values.flatten().shape

In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

fig, axs = plt.subplots(2, sharex=True, sharey=False, figsize=(5,10), gridspec_kw={'height_ratios': [3, 1]})
plt.subplots_adjust(wspace=0, hspace=0)

palette = sns.color_palette("colorblind", 2)
im = sns.violinplot(data=melted,
                    y="Accuracy", x="Protocol", hue="AI Expertise",
                    cut=0, inner='quartile', positions=0, split=True, ax=axs[0])

for violin, alpha, color in zip(axs[0].collections, [0.8, 0.4]*5 + [1, 1], [palette[0]]*6 + [palette[1]]*4 + [palette[0], palette[1]]):
  violin.set_alpha(alpha)
  violin.set_color(color)
#ax = sns.pointplot(x="Protocol", y="Accuracy", data=melted, estimator=np.mean, hue="AI Expertise", join=False, dodge=0.4, color='k', markers='_',)

axs[0].axhline(y = 0.8, color='k', alpha=0.5, label='AI model')
axs[0].set_xlabel('')

handles, labels = axs[0].get_legend_handles_labels()


legend_elements = [#Line2D([0], [0], color='k', alpha=0.5, lw=4, label='AI model'),
                   Patch(facecolor=palette[0], alpha= 0.8, edgecolor='k',label='Hound (SHC)'),
                   Patch(facecolor=palette[0], alpha= 0.4, edgecolor='k',label='Hound (THC)'),
                   Patch(facecolor=palette[0], alpha= 1.0, edgecolor='k',label='Hound'),
                   Patch(facecolor=palette[1], alpha= 0.8, edgecolor='k',label='Ram (SHC)'),
                   Patch(facecolor=palette[1], alpha= 0.4, edgecolor='k',label='Ram (THC)'),
                   Patch(facecolor=palette[1], alpha= 1.0, edgecolor='k',label='Ram')]

axs[0].legend(handles = legend_elements, loc='center',
              bbox_to_anchor=(0.5,1.1), title="Human-AI Collaboration Protocol (MRI Study)", ncol=2)
axs[0].set_ylim(0.59, 1)
axs[0].set_yticks(np.array(range(60, 101, 5))/100)
axs[0].spines['bottom'].set_visible(False)

axs[1].set_xticklabels(melted["Protocol"].unique(), rotation=45, ha="right")
axs[1].errorbar(x=range(len(melted["Protocol"].unique())),
                y=means.values.flatten(), yerr=errors.values.flatten(),
                ls='none', fmt='o', capsize=5, color="red")
axs[1].set_ylim(-0.25, 0.25)
axs[1].yaxis.tick_right()
axs[1].axhline(y = 0.0, color='red', ls='--', alpha=0.5)
axs[1].set_yticks([-0.25, 0.0, 0.25])
axs[1].yaxis.set_tick_params(labelsize=9)
axs[1].set_ylabel("Accuracy Difference", rotation=270, fontsize=9, color="red")
axs[1].yaxis.set_label_position("right")
axs[1].spines['top'].set_visible(False)
axs[1].spines['bottom'].set_color('red')
axs[1].spines['left'].set_color('red')
axs[1].spines['right'].set_color('red')
axs[1].tick_params(axis='y', colors='red')



plt.savefig("MRI_distrib_accs.png", dpi=300, bbox_inches='tight')


In [ ]:
def print_test(table, protocol1, protocol2,
               expertise1=None, expertise2=None, paired=False):
  filter_group1 = table["Protocol"] == protocol1
  filter_expert1 = table["AI Expertise"] == expertise1
  filter1 = np.array([True]*len(table))
  if protocol1 is not None:
    filter1 = filter1 & filter_group1
  if expertise1 is not None:
    filter1 = filter1 & filter_expert1

  filter_group2 = table["Protocol"] == protocol2
  filter_expert2 = table["AI Expertise"] == expertise2
  filter2 = np.array([True]*len(table))
  if protocol2 is not None:
    filter2 = filter2 & filter_group2
  if expertise2 is not None:
    filter2 = filter2 & filter_expert2

  size1 = np.sum(range(len(table[filter1])))
  size2 = np.sum(range(len(table[filter2])))
  size = np.min([size1, size2])

  test = pingouin.wilcoxon if paired else pingouin.mwu
    
  print(table[filter1]["Accuracy"].shape)
  print(table[filter2]["Accuracy"].shape)

  results = test(table[filter1]["Accuracy"], table[filter2]["Accuracy"])
  return results["p-val"], results["RBC"]

In [ ]:
protocols = melted["Protocol"].unique()
expertises = melted["AI Expertise"].unique()

idxnames = protocols#[(n[:-1] + ", " + e + ")") if n != "HD1" else ("HD1 (" + e + ")")  for n in protocols for e in expertises]
print(idxnames)

pvalues = pd.DataFrame(np.ones((6,1)), index=idxnames, columns=["pval"])
effects = pd.DataFrame(np.zeros((6,1)), index=idxnames, columns=["effect"])

for i in range(len(protocols)):
        pr = protocols[i]
        e1 = "SHC"
        e2 = "THC"
        
        p, e = print_test(melted, pr, pr, e1, e2, paired=False)
      
        pvalues.loc[pr,"pval"] = p.values
        effects.loc[pr,"effect"] = e.values
print(pvalues)
print(effects)

In [ ]:
melted[melted['Protocol'] == "HD1"]

In [ ]:
protocols = melted[melted['Protocol'] != "Hound vs Ram"]["Protocol"].unique()
types = {"HD1" : "Hound",
         "HD1-AI-FHD \n(Hound)" : "Hound",
         "HD1-AI-XAI-FHD \n(Hound)": "Hound",
         "AI-FHD \n(Ram)" : "Ram",
         "AI-XAI-FHD \n(Ram)": "Ram"}
expertises = melted["AI Expertise"].unique()

idxnames = protocols#[(n[:-1] + ", " + e + ")") if n != "HD1" else ("HD1 (" + e + ")")  for n in protocols for e in expertises]
print(idxnames)

pvalues = pd.DataFrame(np.ones((5,5)), index=idxnames, columns=idxnames)
effects = pd.DataFrame(np.zeros((5,5)), index=idxnames, columns=idxnames)

for i1 in range(len(protocols)):
  for i2 in range(len(protocols)):
        p1 = protocols[i1]
        p2 = protocols[i2]
        e1 = None#expertises[j1]
        e2 = None#expertises[j2]
        
        print(p1)
        print(p2)
        
        p, e = 0, 0
        if p1 == p2:
          p, e = 1, 0
        else:
          paired = types[p1] == types[p2]
          p, e = print_test(melted[melted['Protocol'] != "Hound vs Ram"], p1, p2, e1, e2, paired=paired)
          p = p.values
          e = e.values
      
        pvalues.loc[p1,p2] = p
        effects.loc[p1,p2] = e
pvalues

In [ ]:
from statsmodels.stats import multitest as stmmulti
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

fig, ax = plt.subplots(figsize=(10,10))

pvalues_corr = pvalues.copy()
pvalues_corr.loc[:,:] = stmmulti.multipletests(pvalues.values.reshape(25,), method="fdr_bh")[1].reshape(5,5)
pvalues_annot = pvalues.applymap("{0:.3f}".format).astype(str) + "\n(" + pvalues_corr.applymap("{0:.3f}".format).astype(str) +")"

cdict = {'red':   [[0.0,  1.0, 1.0],
                   [1.0, 1.0, 1.0]],
         'green': [[0.0,  0.0, 0.0],
                   [0.67, 1.0, 1.0],
                   [1.0, 1.0, 1.0]],
         'blue':  [[0.0,  0.0, 0.0],
                   [0.67, 1.0, 1.0],
                   [1.0, 1.0, 1.0]]}
cmap =  colors.LinearSegmentedColormap('testCmap', segmentdata=cdict, N=2048)



matrix = np.tril(pvalues)
ones = np.triu(np.ones(matrix.shape))
matrix += ones
matrix = pd.DataFrame(matrix, index=pvalues_corr.index, columns = pvalues_corr.index)
mask = np.zeros_like(matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
norm = colors.LogNorm(vmin=0.001, vmax=1)

cax = inset_axes(ax,
                 width="5%",  # width: 40% of parent_bbox width
                 height="100%",  # height: 10% of parent_bbox height
                 bbox_to_anchor=(-0.15, 0, 1, 1),
                 loc="lower left",
                 bbox_transform=ax.transAxes,
                 borderpad=0
                 )
cax.yaxis.set_ticks_position('left')

hm1 = sns.heatmap(matrix, 
                mask=mask, linewidth=1, linecolor='black',
                annot=pvalues_annot, fmt="s",
                cmap=cmap, cbar=True, norm=norm, vmin=0, vmax=1, ax=ax,
                cbar_ax=cax)
colorbar1 = hm1.collections[0].colorbar
colorbar1.set_ticks([0, 0.001,  0.01, 0.05, 0.1, 1])
colorbar1.set_ticklabels([0, "< 0.001", 0.01, 0.05, 0.1, 1])
colorbar1.outline.set_edgecolor('black')
colorbar1.outline.set_linewidth(1)
colorbar1.ax.yaxis.set_ticks_position("left")
colorbar1.ax.set_xlabel('p-value\n(adjusted)')




cdict = {'red':   [[0.0,  1.0, 1.0],
                   [1.0, 0.0, 0.0]],
         'green': [[0.0,  1.0, 1.0],
                   [1.0, 0.0, 0.0]],
         'blue':  [[0.0,  1.0, 1.0],
                   [1.0, 1.0, 1.0]]}
cmap =  colors.LinearSegmentedColormap('testCmap', segmentdata=cdict, N=2048)

matrix = np.triu(np.abs(effects))
matrix = pd.DataFrame(matrix, index=effects.index, columns = effects.index)
mask = np.zeros_like(matrix, dtype=np.bool)
mask[np.tril_indices_from(mask)] = True
hm2 = sns.heatmap(matrix, 
                mask=mask, linewidth=1, linecolor='black',
                annot=True, fmt=".2f",
                cmap=cmap, cbar=True, vmin=0, vmax=1, ax=ax)

colorbar2 = hm2.collections[1].colorbar
#colorbar2.set_ticks([0, 0.2, 0.4, 0.6, 0.8, 1])
#colorbar2.set_ticklabels([0, 0.2, 0.4, 0.6, 0.8, 1])
colorbar2.outline.set_edgecolor('black')
colorbar2.outline.set_linewidth(1)
colorbar2.ax.set_xlabel('Effect size\n(RBC)')






plt.savefig("MRI-info.png", dpi=300, bbox_inches="tight")

In [ ]:
print("AI-FHD %.2f +- %.2f" % 
      (data["Accuracy (Ram, without AM)"].mean(),
       1.96*data["Accuracy (Ram, without AM)"].std()/np.sqrt(12)))
print("AI-XAI-FHD %.2f +- %.2f" % 
      (data["Accuracy (Ram, with AM)"].mean(),
       1.96*data["Accuracy (Ram, with AM)"].std()/np.sqrt(12)))
print("HD1 %.2f +- %.2f" % 
      (data["Accuracy (Hound, before AI support)"].mean(),
       1.96*data["Accuracy (Hound, before AI support)"].std()/np.sqrt(12)))
print("HD1-AI-FHD %.2f +- %.2f" % 
      (data["Accuracy (Hound, without AM)"].mean(),
       1.96*data["Accuracy (Hound, without AM)"].std()/np.sqrt(12)))
print("HD1-AI-XAI-FHD %.2f +- %.2f" % 
      (data["Accuracy (Hound, with AM)"].mean(),
       1.96*data["Accuracy (Hound, with AM)"].std()/np.sqrt(12)))
print()

print("AI-FHD (SHC) %.2f +- %.2f" % 
      (data[data["Group"] == "SHC"]["Accuracy (Ram, without AM)"].mean(),
       1.96*data[data["Group"] == "SHC"]["Accuracy (Ram, without AM)"].std()/np.sqrt(6)))
print("AI-XAI-FHD (SHC) %.2f +- %.2f" % 
      (data[data["Group"] == "SHC"]["Accuracy (Ram, with AM)"].mean(),
       1.96*data[data["Group"] == "SHC"]["Accuracy (Ram, with AM)"].std()/np.sqrt(6)))
print("HD1 (SHC) %.2f +- %.2f" % 
      (data[data["Group"] == "SHC"]["Accuracy (Hound, before AI support)"].mean(),
       1.96*data[data["Group"] == "SHC"]["Accuracy (Hound, before AI support)"].std()/np.sqrt(6)))
print("HD1-AI-FHD (SHC) %.2f +- %.2f" % 
      (data[data["Group"] == "SHC"]["Accuracy (Hound, without AM)"].mean(),
       1.96*data[data["Group"] == "SHC"]["Accuracy (Hound, without AM)"].std()/np.sqrt(6)))
print("HD1-AI-XAI-FHD (SHC) %.2f +- %.2f" % 
      (data[data["Group"] == "SHC"]["Accuracy (Hound, with AM)"].mean(),
       1.96*data[data["Group"] == "SHC"]["Accuracy (Hound, with AM)"].std()/np.sqrt(6)))
print()

print("AI-FHD (THC) %.2f +- %.2f" % 
      (data[data["Group"] == "THC"]["Accuracy (Ram, without AM)"].mean(),
       1.96*data[data["Group"] == "THC"]["Accuracy (Ram, without AM)"].std()/np.sqrt(6)))
print("AI-XAI-FHD (THC) %.2f +- %.2f" % 
      (data[data["Group"] == "THC"]["Accuracy (Ram, with AM)"].mean(),
       1.96*data[data["Group"] == "THC"]["Accuracy (Ram, with AM)"].std()/np.sqrt(6)))
print("HD1 (THC) %.2f +- %.2f" % 
      (data[data["Group"] == "THC"]["Accuracy (Hound, before AI support)"].mean(),
       1.96*data[data["Group"] == "THC"]["Accuracy (Hound, before AI support)"].std()/np.sqrt(6)))
print("HD1-AI-FHD (THC) %.2f +- %.2f" % 
      (data[data["Group"] == "THC"]["Accuracy (Hound, without AM)"].mean(),
       1.96*data[data["Group"] == "THC"]["Accuracy (Hound, without AM)"].std()/np.sqrt(6)))
print("HD1-AI-XAI-FHD (THC) %.2f +- %.2f" % 
      (data[data["Group"] == "THC"]["Accuracy (Hound, with AM)"].mean(),
       1.96*data[data["Group"] == "THC"]["Accuracy (Hound, with AM)"].std()/np.sqrt(6)))

In [ ]:
diff_A = np.array(data[data["Group"] == "SHC"]["Accuracy (Ram, with AM)"]) - np.array(data[data["Group"] == "SHC"]["Accuracy (Ram, without AM)"])

diff_B = np.array(data[data["Group"] == "THC"]["Accuracy (Ram, with AM)"]) - np.array(data[data["Group"] == "THC"]["Accuracy (Ram, without AM)"])

diff_agg = np.array(data["Accuracy (Ram, with AM)"]) - np.array(data["Accuracy (Ram, without AM)"])

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(x=data[data["Group"] == "SHC"]["Accuracy (Ram, without AM)"],
                y=diff_A, color="black", edgecolor="black", alpha=1, label="SHC")
sns.scatterplot(x=data[data["Group"] == "THC"]["Accuracy (Ram, without AM)"],
                y=diff_B, color="white", edgecolor="black", alpha=1, label="THC")

vals = np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plt.fill_between(vals, 1 - vals, alpha=0.3, color="blue" )
plt.fill_between(vals, - vals, alpha=0.3, color="red" )

plt.axhline(np.mean(diff_agg), color="sienna")
plt.axhline(np.mean(diff_agg) + 1.96*np.std(diff_agg)/np.sqrt(len(diff_agg)), color="sienna", alpha=0.5)
plt.axhline(np.mean(diff_agg) - 1.96*np.std(diff_agg)/np.sqrt(len(diff_agg)), color="sienna", alpha=0.5)
plt.xlim(0,1)
plt.ylim(-1,1)
plt.xlabel("AI-FHD")
plt.ylabel("Benefit of XAI support ( = AI-XAI-FHD - AI-FHD)")

plt.savefig("XAI_vs_NoXAI_Ram_bd.png", dpi=300, bbox_inches="tight")

In [ ]:
diff_A = np.array(data[data["Group"] == "SHC"]["Accuracy (Hound, with AM)"]) - np.array(data[data["Group"] == "SHC"]["Accuracy (Hound, without AM)"])
diff_B = np.array(data[data["Group"] == "THC"]["Accuracy (Hound, with AM)"]) - np.array(data[data["Group"] == "THC"]["Accuracy (Hound, without AM)"])
diff_aggAB = np.array(data["Accuracy (Hound, with AM)"]) - np.array(data["Accuracy (Hound, without AM)"])

diff_C = np.array(data[data["Group"] == "SHC"]["Accuracy (Hound, with AM)"]) - np.array(data[data["Group"] == "SHC"]["Accuracy (Hound, before AI support)"])
diff_D = np.array(data[data["Group"] == "THC"]["Accuracy (Hound, with AM)"]) - np.array(data[data["Group"] == "THC"]["Accuracy (Hound, before AI support)"])
diff_aggCD = np.array(data["Accuracy (Hound, with AM)"]) - np.array(data["Accuracy (Hound, before AI support)"])

diff_E = np.array(data[data["Group"] == "SHC"]["Accuracy (Hound, without AM)"]) - np.array(data[data["Group"] == "SHC"]["Accuracy (Hound, before AI support)"])
diff_F = np.array(data[data["Group"] == "THC"]["Accuracy (Hound, without AM)"]) - np.array(data[data["Group"] == "THC"]["Accuracy (Hound, before AI support)"])
diff_aggEF = np.array(data["Accuracy (Hound, without AM)"]) - np.array(data["Accuracy (Hound, before AI support)"])

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(x=data[data["Group"] == "SHC"]["Accuracy (Hound, without AM)"],
                y=diff_A, color="black",  edgecolor="black", alpha=1, label="SHC")
sns.scatterplot(x=data[data["Group"] == "THC"]["Accuracy (Hound, without AM)"],
                y=diff_B, color="white",  edgecolor="black", alpha=1, label="THC")

vals = np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plt.fill_between(vals, 1 - vals, alpha=0.3, color="blue" )
plt.fill_between(vals, - vals, alpha=0.3, color="red" )

plt.axhline(np.mean(diff_aggAB), color="sienna")
plt.axhline(np.mean(diff_aggAB) + 1.96*np.std(diff_aggAB)/np.sqrt(len(diff_aggAB)), color="sienna", alpha=0.5)
plt.axhline(np.mean(diff_aggAB) - 1.96*np.std(diff_aggAB)/np.sqrt(len(diff_aggAB)), color="sienna", alpha=0.5)
plt.xlim(0,1)
plt.ylim(-1,1)
plt.xlabel("HD1-AI-FHD")
plt.ylabel("Benefit of XAI support ( = HD1-AI-XAI-FHD - HD1-AI-FHD)")

plt.savefig("XAI_vs_noXAI_Hound_bd.png", dpi=300, bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(x=data[data["Group"] == "SHC"]["Accuracy (Hound, before AI support)"],
                y=diff_C, color="black",  edgecolor="black", alpha=1, label="SHC")
sns.scatterplot(x=data[data["Group"] == "THC"]["Accuracy (Hound, before AI support)"],
                y=diff_D, color="white",  edgecolor="black", alpha=1, label="THC")

vals = np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plt.fill_between(vals, 1 - vals, alpha=0.3, color="blue" )
plt.fill_between(vals, - vals, alpha=0.3, color="red" )

plt.axhline(np.mean(diff_aggCD), color="sienna")
plt.axhline(np.mean(diff_aggCD) + 1.96*np.std(diff_aggCD)/np.sqrt(len(diff_aggCD)), color="sienna", alpha=0.5)
plt.axhline(np.mean(diff_aggCD) - 1.96*np.std(diff_aggCD)/np.sqrt(len(diff_aggCD)), color="sienna", alpha=0.5)
plt.xlim(0,1)
plt.ylim(-1,1)
plt.xlabel("HD1")
plt.ylabel("Benefit of AI+XAI support ( = HD1-AI-XAI-FHD - HD1)")

plt.savefig("XAI_vs_noAI_Hound_bd.png", dpi=300, bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(x=data[data["Group"] == "SHC"]["Accuracy (Hound, before AI support)"],
                y=diff_E, color="black",  edgecolor="black", alpha=1, label="SHC")
sns.scatterplot(x=data[data["Group"] == "THC"]["Accuracy (Hound, before AI support)"],
                y=diff_F, color="white",  edgecolor="black", alpha=1, label="THC")

vals = np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plt.fill_between(vals, 1 - vals, alpha=0.3, color="blue" )
plt.fill_between(vals, - vals, alpha=0.3, color="red" )

plt.axhline(np.mean(diff_aggEF), color="sienna")
plt.axhline(np.mean(diff_aggEF) + 1.96*np.std(diff_aggEF)/np.sqrt(len(diff_aggEF)), color="sienna", alpha=0.5)
plt.axhline(np.mean(diff_aggEF) - 1.96*np.std(diff_aggEF)/np.sqrt(len(diff_aggEF)), color="sienna", alpha=0.5)
plt.xlim(0,1)
plt.ylim(-1,1)
plt.xlabel("HD1")
plt.ylabel("Benefit of AI support ( = HD1-AI-FHD - HD1)")

plt.savefig("AI_vs_noAI_Hound_bd.png", dpi=300, bbox_inches="tight")

In [ ]:
data.columns
data_part = data[["Group",
                  "Positive AI Reliance No AM","Positive AI Reliance AM",
                  "Negative Self-Reliance No AM","Negative Self-Reliance AM",
                  "Negative AI Reliance No AM","Negative AI Reliance AM",
                  "Positive Self-Reliance No AM","Positive Self-Reliance AM",
                  "Positive AI Avoidance No AM", "Positive AI Avoidance AM",
                  "Negative AI Acceptance No AM", "Negative AI Acceptance AM",
                  "Positive AI Acceptance No AM", "Positive AI Acceptance AM",
                  "Negative AI Avoidance No AM", "Negative AI Avoidance AM"]].copy()
# = data_part[data_part["Group"] == "SHC"]
data_part

In [ ]:
RAIR_NoXAI = data_part["Positive AI Reliance No AM"]/(data_part["Positive AI Reliance No AM"] + data_part["Negative Self-Reliance No AM"])
RAIR_XAI = data_part["Positive AI Reliance AM"]/(data_part["Positive AI Reliance AM"] + data_part["Negative Self-Reliance AM"])

RSR_NoXAI = data_part["Positive Self-Reliance No AM"]/(data_part["Negative AI Reliance No AM"] + data_part["Positive Self-Reliance No AM"])
RSR_XAI = data_part["Positive Self-Reliance AM"]/(data_part["Negative AI Reliance AM"] + data_part["Positive Self-Reliance AM"])

RAID_NoXAI = data_part["Positive AI Avoidance No AM"]/(data_part["Positive AI Avoidance No AM"] + data_part["Negative AI Acceptance No AM"])
RAID_XAI = data_part["Positive AI Avoidance AM"]/(data_part["Positive AI Avoidance AM"] + data_part["Negative AI Acceptance AM"])

RST_NoXAI = data_part["Positive AI Acceptance No AM"]/(data_part["Negative AI Avoidance No AM"] + data_part["Positive AI Acceptance No AM"])
RST_XAI = data_part["Positive AI Acceptance AM"]/(data_part["Negative AI Avoidance AM"] + data_part["Positive AI Acceptance AM"])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

def trust_diagram(RAIRs, RSRs, RAIDs, RSTs, labels, confidence=None, filename="trust-diagram"):
  fig, axs = plt.subplots(ncols=2, sharex=False, sharey=False, figsize=(10,5))
  plt.subplots_adjust(wspace=0.05, hspace=0)

  palette = sns.color_palette("colorblind", 2)

  for i in range(len(RAIDs)):
    if confidence == 'ellipse':
      el = Ellipse(xy = (RAIDs[i].mean(), RSRs[i].mean()),
          width=2*1.96*RAIDs[i].std()/np.sqrt(len(RAIDs[i])),
          height=2*1.96*RSRs[i].std()/np.sqrt(len(RSRs[i])),
          label = labels[i], edgecolor=palette[i], facecolor=palette[i], alpha=0.6)
      
      axs[0].add_patch(el)
    elif confidence == 'error':
      axs[0].errorbar(RAIDs[i].mean(), RSRs[i].mean(),
                      xerr=1.96*RAIDs[i].std()/np.sqrt(len(RAIDs[i])),
                      yerr=1.96*RSRs[i].std()/np.sqrt(len(RSRs[i])),
                      label=labels[i], color=palette[i])
    else:
      axs[0].scatter(RAIDs[i].mean(), RSRs[i].mean(), label=labels[i], color=palette[i])

  axs[0].axhline(y = 0.5, color='k', ls='--', alpha=0.5)
  axs[0].axvline(x = 0.5, color='k', ls='--', alpha=0.5)
  axs[0].fill_between([-0.02,1.02], -0.02, 0.5, color='red', alpha=0.2, label="Automation Bias") #RSR Basso
  axs[0].fill_betweenx([-0.02,1.02], -0.02, 0.5, color='blue', alpha=0.2, label="Automation Complacency") #RAIR Basso

  axs[0].set_xlim(-0.02, 1.02)
  axs[0].set_ylim(-0.02, 1.02)
  axs[0].set_xlabel("Relative Beneficial Over-distrust (RBOD)")
  axs[0].set_ylabel("Relative Beneficial Distrust (RBD)")
  axs[0].legend(loc='center', bbox_to_anchor=(0.5,1.1), ncol=2)

  for i in range(len(RSTs)):
    if confidence == 'ellipse':
      el = Ellipse(xy = (RAIRs[i].mean(), RSTs[i].mean()),
          width=2*1.96*RAIRs[i].std()/np.sqrt(len(RAIRs[i])),
          height=2*1.96*RSTs[i].std()/np.sqrt(len(RSTs[i])),
          label = labels[i], edgecolor=palette[i], facecolor=palette[i], alpha=0.6)
      
      axs[1].add_patch(el)
    elif confidence == 'error':
      axs[1].errorbar(RAIRs[i].mean(), RSTs[i].mean(),
                      xerr=1.96*RAIRs[i].std()/np.sqrt(len(RAIRs[i])),
                      yerr=1.96*RSTs[i].std()/np.sqrt(len(RSTs[i])),
                      label=labels[i], color=palette[i])
    else:
      axs[1].scatter(RAIRs[i].mean(), RSTs[i].mean(), label=labels[i], color=palette[i])

  axs[1].axhline(y = 0.5, color='k', ls='--', alpha=0.5)
  axs[1].axvline(x = 0.5, color='k', ls='--', alpha=0.5)
  axs[1].fill_between([-0.02,1.02], -0.02, 0.5, color='red', alpha=0.2, label="Algorithmic Aversion") #RAID basso
  axs[1].fill_betweenx([-0.02,1.02], -0.02, 0.5, color='blue', alpha=0.2, label="Conservatism Bias") #RST basso

  axs[1].set_xlim(-0.02, 1.02)
  axs[1].set_ylim(-0.02, 1.02)
  axs[1].set_xlabel("Relative Beneficial Over-trust (RBOT)")
  axs[1].set_ylabel("Relative Beneficial Trust (RBT)")
  axs[1].yaxis.tick_right()
  axs[1].yaxis.set_label_position("right")
  axs[1].invert_xaxis()
  axs[1].legend(loc='center', bbox_to_anchor=(0.5,1.1), ncol=2)

  plt.savefig(filename + ".png", dpi=300, bbox_inches="tight")

In [ ]:
trust_diagram([RAIR_NoXAI,RAIR_XAI], [RSR_NoXAI, RSR_XAI],
                          [RAID_NoXAI, RAID_XAI], [RST_NoXAI, RST_XAI],
                          ["HD1-AI-FHD", "HD1-AI-XAI-FHD"],
                          confidence="error")

In [ ]:
trust_diagram([], [],
                          [], [],
                          [],
                          confidence="error",
                          filename="trust-diagram-empty")